In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import random
import math
import os
import shutil
import time

CRAWL_PROFILE = True
CRAWL_ARTICLE = True

pd.set_option('display.max_colwidth', None)
home_url='https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/'
default_url='https://pureportal.coventry.ac.uk'
articles_inputFile = "C:/Users/ramna/DataScience/IR/7071\Scraped_Articles_Details.csv"

# Variables 

In [27]:
queue= [home_url]
already_visited=[]
total_entries= 0
last_page_entries =0
profile_id = 1
document_id =1
data_table = {}
article_table = {}

# Fetching home page data details for Profile

In [28]:
#Input : page-class:`Response <Response>` object from GET request. 
# Update profile information in the global data_table
def home_page_details(page):
    global default_url
    global total_entries
    global queue
    global last_page_entries
    global data_table
    global profile_id

    soup = BeautifulSoup(page.content, 'html.parser')
    users = soup.find_all('div',attrs={'class':'result-container'})
    no_of_entries_in_page = 0
    for user in users:
        profile_name=user.find('span').text
        link = user.find('a')['href']
        no_of_entries_in_page +=1

        data_table[profile_id] = [profile_name,link]
        
        profile_id =profile_id+1

    if last_page_entries == 0:
        last_page_entries = no_of_entries_in_page

    if no_of_entries_in_page <last_page_entries : # We have reached to last page
        total_entries=total_entries+no_of_entries_in_page
    else:    
        total_entries=total_entries+no_of_entries_in_page
        next_page=soup.find('a',attrs={'class':'nextLink'})['href']
        next_page_link=default_url+next_page
        queue.append(next_page_link)
        last_page_entries = no_of_entries_in_page

# Fetching next page data details for Article

In [29]:
# Input - Profile link
# output - update global article_table with all the details related with the profile 
def article_details(profile_url):
    global article_table
    global document_id
    start = 0
    page_size=20    
    #access url of profile 
    url = "/publications/"
    #print("Url ::", profile_url+url)
    page =requests.get(profile_url+url)   
    
    if page.status_code != 200:
        print("Failed to access url..[ERROR_CODE]:", page.status_code)
        print("Profile Url : ",profile_url)
        raise Exception("Error loading page..")

    else:
        #read the page HTML content
        soup = BeautifulSoup(page.content, 'html.parser')
        #find  all the articles on single page
        name = soup.find('h1').text
        titles = soup.find_all('h3', attrs={'class': 'title'})
#       next_page=soup.find('div',attrs={'class':'view-all align-center'})
        while len(titles) > 0: # NOT NULL
            no_of_citations = soup.find_all('div',attrs={'class':'metric scopus-citations'})
            for title, citation in zip(titles,no_of_citations):
                
                profile_name = name
                title_name = title.find('a',attrs={'class': 'link'}).text
                co_authors = soup.find('a',attrs={'class': 'link person'}).text
                title_link = title.find('a')['href']
                no_Of_Citations = citation.find('a').text
                
                article_table[document_id] = [profile_name,title_name,co_authors,title_link,no_Of_Citations]

                document_id=document_id+1 # increment the document id for next item
            
#           nxt_pge = next_page.find('a', attrs={'class': 'portal_link btn-primary btn-large'})['href']
#           new_page_link = requests.get(default_url+nxt_pge)
#           soup = BeautifulSoup(new_page_link.content, 'html.parser')
            titles = soup.find_all('h3', attrs={'class': 'title1'})            

In [16]:
page = requests.get(home_url)
print(page.status_code)# Test code


200


In [6]:
#hard stop after 500 loops to avoid any server overload due to infinite crawling
stop = 100 # Use only for top level of scraping 
start_time = time.time() # measure time taken to scrape data 
failure=False



if CRAWL_PROFILE:
    #1.1 Crawl through all the pages and extract profile links into a list
    print("\nStep 1 :: Fetching all the pages for profile links into CSV File")
    print("\nHome URL : ",home_url)
    global data_table
    col_names_profile=['Profile_Name','Profile_Link']


    while len(queue)!=0 and stop >0:
        try:
            random_time=random.randint(0,1) # genrate random time wait in sec
            #retrieve HTML content of the page
        #     print("url : ",queue[0])
            page =requests.get(queue[0])

            #extract all the requried info from the page
            if page.status_code==200:
                home_page_details(page)
            else:
                print("Failed to load page [ERROR_CODE]: ",page.status_code)
                print("Page Url : ",queue[0])

            #Try next link -> pop out the first page from main queue::FIFO and add into already visited url list
            #print(len(queue))
            if len(queue)>0 :
                already_visited.append(queue.pop(0))
            stop = stop-1
            time.sleep(random_time)  # add delay of 1s before visiting next page
        except:
            print("Inside Step 1 fetching....something went wrong..Exiting\n")
            failure=True
            break

    stop_time = time.time()        
    if failure==False:
        time_taken = stop_time-start_time
        
        print("\nSaving all the crawling pages for profiles links into CSV File.")
        #Write profile data into file
        

        data_frame=pd.DataFrame.from_dict(data_table,orient ='index',columns=col_names_profile)
        data_frame.index.rename('Profile_Id', inplace=True)
    
        file_exists = os.path.isfile("Scraped_Profile_Details.csv")
        if file_exists: #backup the file
            shutil.copy("Scraped_Profile_Details.csv", "Scraped_Profile_Details_Last.csv")  
        
        data_frame.to_csv("Scraped_Profile_Details.csv")
        
        print("\nSaved Successfully")         
    else:
        time_taken = stop_time-start_time
        
        print("\nSaving all the crawling pages for profiles links into CSV File.")
        #Write profile data into file
        #save_data_profile()  

        data_frame=pd.DataFrame.from_dict(data_table,orient ='index',columns=col_names_profile)
        data_frame.index.rename('Profile_Id', inplace=True)
    
        file_exists = os.path.isfile("Scraped_Profile_Details.csv")
        if file_exists: #backup the file
            shutil.copy("Scraped_Profile_Details.csv", "Scraped_Profile_Details_Last.csv")  
        
        data_frame.to_csv("Scraped_Profile_Details.csv") 
        
        print("Step 1: fetching of profiles failed")

if CRAWL_ARTICLE:    
    start_time = time.time() # measure time taken to scrape data 
    #1.2 Crawl through all the profiles link now to extract articles for each users
    profile_df =pd.read_csv("Scraped_Profile_Details.csv",header='infer')
    profile_queue = profile_df['Profile_Link'].values # assign the profile_queue with list of all the pofile link ::FIFO
    head_profile_url = profile_queue[0]
    stop = len(profile_queue)
    global article_table
    col_names_article=['Profile_Name','Title','Co_Authors','Title_Link','No_Of_Citations']

    print("\nStep 2 :: Fetching all the Articles From Profile URL into CSV File")
    print("\nFirst Page of Profile URL : ",head_profile_url)
    print("\nTotal Page Fetched: ",len(profile_queue))

    while len(profile_queue) != 0 and stop >0:
        try:
            stop = stop-1
            #random_time=random.randint(0,1) # genrate random time wait in sec

               # Crawl to individual user profile and extract their published articles details
            article_details(head_profile_url)
            #print("Processsing done , pending ", len(profile_queue)-1)
                #pop out the first link from the queue and assign second link as first item
            if len(profile_queue)!=1:
                profile_queue = profile_queue[1:]
                #assign head of queue to head_profile_url for next crawl
                head_profile_url = profile_queue[0]
            else:
                profile_queue = [] # All the links are crawled
                
            time.sleep(random_time)  # add delay of 1s before visiting next page
        except:
            failure = True
            break

    stop_time = time.time()
    if failure == False:
        time_taken = stop_time-start_time

        print("Nothing to process. Queue Len : ",len(queue))
        print("No.Of Entries extracted : ", total_entries)
        print("\nCompleted before hard stop, Iterations still left before hard stop(max 500 iterations) : ",stop)

        #Write published article data into file
        print("\nSaving all the Articles details into CSV File")    

        data_frame=pd.DataFrame.from_dict(article_table,orient='index',columns=col_names_article)
        data_frame.index.rename('Document_Id', inplace=True)
    
        file_exists = os.path.isfile("Scraped_Articles_Details.csv")
        if file_exists: #backup the file
            shutil.copy("Scraped_Articles_Details.csv", "Scraped_Articles_Details_Last.csv") 
        
        data_frame.to_csv("Scraped_Articles_Details.csv")
        
        print("\nSaved Successfully.")  
    else:
        time_taken = stop_time-start_time
        
        
        print("\nSaving all the Articles details into CSV File")  
        data_frame=pd.DataFrame.from_dict(article_table,orient='index',columns=col_names_article)
        data_frame.index.rename('Document_Id', inplace=True)
    
        file_exists = os.path.isfile("Scraped_Articles_Details.csv")
        if file_exists: #backup the file
            shutil.copy("Scraped_Articles_Details.csv", "Scraped_Articles_Details_Last.csv") 
        
        data_frame.to_csv("Scraped_Articles_Details.csv")
        
        print("\nSaved Successfully")
    


Step 1 :: Fetching all the pages for profile links into CSV File

Home URL :  https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/

Saving all the crawling pages for profiles links into CSV File.

Saved Successfully

Step 2 :: Fetching all the Articles From Profile URL into CSV File

Profile URL :  https://pureportal.coventry.ac.uk/en/persons/mohammadreza-abbassi-monjezi

Total Iteration Page:  2185

Saving all the Articles details into CSV File

Saved Successfully
